In [9]:
import csv
import os

dataset_path_1 = '/Users/patron/Developer/SQL Project/dataset/US_flights_2023.csv'

# Check if the file exists before attempting to open
if not os.path.exists(dataset_path_1):
    print(f"Error: The file '{dataset_path_1}' was not found. Please ensure the path is correct.")
else:
    try:
        with open(dataset_path_1, 'r') as file:
            reader = csv.reader(file)

            # Read the header row
            headers = next(reader)

            # Count the remaining data rows
            row_count = sum(1 for row in reader)

            # Add 1 for the header row to get total rows
            total_rows = row_count + 1

            # Number of columns is the length of the headers
            total_columns = len(headers)

            print("\n--- Dataset Description (using csv module) ---")
            print(f"Total Number of Rows (including header): {total_rows}")
            print(f"Total Number of Data Rows ( excluding header): {row_count}")
            print(f"Total Number of Columns: {total_columns}")
            print("Field Names (Columns):")
            for i, header in enumerate(headers):
                print(f"  {i+1}. {header}")
            print(f"Dataset Dimensions (Rows, Columns): ({total_rows}, {total_columns})")

    except Exception as e:
        print(f"An error occurred while reading the dataset with csv module: {e}")


--- Dataset Description (using csv module) ---
Total Number of Rows (including header): 6743405
Total Number of Data Rows ( excluding header): 6743404
Total Number of Columns: 24
Field Names (Columns):
  1. FlightDate
  2. Day_Of_Week
  3. Airline
  4. Tail_Number
  5. Dep_Airport
  6. Dep_CityName
  7. DepTime_label
  8. Dep_Delay
  9. Dep_Delay_Tag
  10. Dep_Delay_Type
  11. Arr_Airport
  12. Arr_CityName
  13. Arr_Delay
  14. Arr_Delay_Type
  15. Flight_Duration
  16. Distance_type
  17. Delay_Carrier
  18. Delay_Weather
  19. Delay_NAS
  20. Delay_Security
  21. Delay_LastAircraft
  22. Manufacturer
  23. Model
  24. Aicraft_age
Dataset Dimensions (Rows, Columns): (6743405, 24)


In [10]:
dataset_path_2 = '/Users/patron/Developer/SQL Project/dataset/weather_meteo_by_airport.csv'

# Check if the file exists before attempting to open
if not os.path.exists(dataset_path_1):
    print(f"Error: The file '{dataset_path_2}' was not found. Please ensure the path is correct.")
else:
    try:
        with open(dataset_path_2, 'r') as file:
            reader = csv.reader(file)

            # Read the header row
            headers = next(reader)

            # Count the remaining data rows
            row_count = sum(1 for row in reader)

            # Add 1 for the header row to get total rows
            total_rows = row_count + 1

            # Number of columns is the length of the headers
            total_columns = len(headers)

            print("\n--- Dataset Description (using csv module) ---")
            print(f"Total Number of Rows (including header): {total_rows}")
            print(f"Total Number of Data Rows (excluding header): {row_count}")
            print(f"Total Number of Columns: {total_columns}")
            print("Field Names (Columns):")
            for i, header in enumerate(headers):
                print(f"  {i+1}. {header}")
            print(f"Dataset Dimensions (Rows, Columns): ({total_rows}, {total_columns})")

    except Exception as e:
        print(f"An error occurred while reading the dataset with csv module: {e}")


--- Dataset Description (using csv module) ---
Total Number of Rows (including header): 132861
Total Number of Data Rows (excluding header): 132860
Total Number of Columns: 10
Field Names (Columns):
  1. time
  2. tavg
  3. tmin
  4. tmax
  5. prcp
  6. snow
  7. wdir
  8. wspd
  9. pres
  10. airport_id
Dataset Dimensions (Rows, Columns): (132861, 10)


In [12]:
import os, psycopg2
from dotenv import load_dotenv, find_dotenv

_ = load_dotenv(find_dotenv())

In [13]:
conn = psycopg2.connect(
    dbname=os.getenv("POSTGRES_DB"),
    user=os.getenv("POSTGRES_USER"),
    password=os.getenv("POSTGRES_PASSWORD"),
    host=os.getenv("POSTGRES_HOST"),
    port=os.getenv("POSTGRES_PORT"),
    # sslmode=os.getenv("PGSSLMODE")
)

In [14]:
# import psycopg2
# from psycopg2 import sql

# dataset_path_1 = '/content/drive/MyDrive/Data Set/US_flights_2023.csv'
# dataset_path_2 = '/content/drive/MyDrive/Data Set/weather_meteo_by_airport.csv'

cur = conn.cursor()

# ---------- 1) CREATE TABLES ----------

ddl = """
-- STAGING TABLES
CREATE TABLE IF NOT EXISTS flights_raw (
    FlightDate       TEXT,
    Day_Of_Week      TEXT,
    Airline          TEXT,
    Tail_Number      TEXT,
    Dep_Airport      TEXT,
    Dep_CityName     TEXT,
    DepTime_label    TEXT,
    Dep_Delay        TEXT,
    Dep_Delay_Tag    TEXT,
    Dep_Delay_Type   TEXT,
    Arr_Airport      TEXT,
    Arr_CityName     TEXT,
    Arr_Delay        TEXT,
    Arr_Delay_Type   TEXT,
    Flight_Duration  TEXT,
    Distance_type    TEXT,
    Delay_Carrier    TEXT,
    Delay_Weather    TEXT,
    Delay_NAS        TEXT,
    Delay_Security   TEXT,
    Delay_LastAircraft TEXT,
    Manufacturer     TEXT,
    Model            TEXT,
    Aicraft_age      TEXT
);

CREATE TABLE IF NOT EXISTS weather_raw (
    time       TEXT,
    tavg       TEXT,
    tmin       TEXT,
    tmax       TEXT,
    prcp       TEXT,
    snow       TEXT,
    wdir       TEXT,
    wspd       TEXT,
    pres       TEXT,
    airport_id TEXT
);

-- DIMENSIONS & FACTS
CREATE TABLE IF NOT EXISTS airports (
    airport_code   TEXT PRIMARY KEY,
    city_name      TEXT,
    state          TEXT,
    country        TEXT
);

CREATE TABLE IF NOT EXISTS airlines (
    airline_code   TEXT PRIMARY KEY
);

CREATE TABLE IF NOT EXISTS aircrafts (
    tail_number    TEXT PRIMARY KEY,
    manufacturer   TEXT,
    model          TEXT,
    aircraft_age   NUMERIC
);

CREATE TABLE IF NOT EXISTS flights (
    flight_id          BIGSERIAL PRIMARY KEY,
    flight_date        DATE NOT NULL,
    day_of_week        SMALLINT,
    airline_code       TEXT REFERENCES airlines(airline_code),
    tail_number        TEXT REFERENCES aircrafts(tail_number),
    dep_airport_code   TEXT REFERENCES airports(airport_code),
    arr_airport_code   TEXT REFERENCES airports(airport_code),
    dep_cityname       TEXT,
    arr_cityname       TEXT,
    dep_time_label     TEXT,
    dep_delay          NUMERIC,
    dep_delay_tag      TEXT,
    dep_delay_type     TEXT,
    arr_delay          NUMERIC,
    arr_delay_type     TEXT,
    flight_duration    NUMERIC,
    distance_type      TEXT,
    delay_carrier      NUMERIC,
    delay_weather      NUMERIC,
    delay_nas          NUMERIC,
    delay_security     NUMERIC,
    delay_lastaircraft NUMERIC
);

CREATE TABLE IF NOT EXISTS weather_observations (
    airport_code   TEXT REFERENCES airports(airport_code),
    obs_date       DATE,
    tavg           NUMERIC,
    tmin           NUMERIC,
    tmax           NUMERIC,
    prcp           NUMERIC,
    snow           NUMERIC,
    wdir           NUMERIC,
    wspd           NUMERIC,
    pres           NUMERIC,
    PRIMARY KEY (airport_code, obs_date)
);
"""

cur.execute(ddl)
conn.commit()
print("Tables created.")


Tables created.
